# Corotation radii measurements distributions

In [1]:
import numpy as np
import math
import pandas as pd
import os
import matplotlib.image as mpimg
import pickle
import matplotlib.patches as mpatches
import pylab as plt
from astroquery.ned import Ned
from shapely.geometry import Point, LineString
from shapely.geometry.polygon import Polygon
import shapely
import warnings
import seaborn as sns
from astroquery.simbad import Simbad
from urllib.parse import urlencode
import aplpy
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [2]:
import matplotlib.pyplot as plt
plt.style.use('classic')
import matplotlib
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('font',**{'family':'serif','serif':['PT Serif']})
rc('text', usetex=True)
matplotlib.rcParams['ps.useafm'] = True
matplotlib.rcParams['pdf.use14corefonts'] = True
matplotlib.rcParams['text.usetex'] = True

In [4]:
df = pd.read_csv('./../total_crs.csv',index_col=0)

In [3]:
method_color = {'F-B' : '#c2bdcc',
 'P-D' : '#5a585f',
 'P-D(bar)' : '#5a585f',
 'SNs' : 'lightsalmon',
 'T-W' : '#3573b0',
 'T-W(bar)' : '#3573b0',
 'bar-torque(bar)' : '#90b059',
 'bar-torque' : '#90b059',
 'gaps(bar)' : '#b089af',
 'gaps' : '#b089af',
 'metallicity' : '#318a61',               
 'model' : '#803c88',
 'model(bar)' : '#803c88',
 'morph' : '#f0e330',                         
 'offset' : '#e15a31',
 'potential-density' : '#53e1c4',
 'rings(bar)' : '#e1a292',
 'rings' : '#e1a292',
 'widths' : '#e1a236'}

In [5]:
def plot_cr_colorlegend(gal_name=None, ax=None, loc=None, fontsize=None, used_methods=None, bars=None):
    if used_methods is None:
        methods = []
        for ind, row in df.iterrows():
            methods.append(row['method'])
        methods = set(methods)
    else:
        for ind,val in enumerate(bars):
            if val=='True':
                meth=used_methods[ind]
                used_methods[ind] = meth+'(bar)'
            methods = set(used_methods)
                
    patches = []
    
    for ind,method in enumerate(methods):
        patch = mpatches.Patch(color=method_color[method], label=method, alpha=0.5)
        patches.append(patch)
    if fontsize is None:
        ax.legend(handles=patches, loc=loc)
        ax.legend(handles=patches, bbox_to_anchor=(1.01,1.2), loc="upper left", 
          fancybox=True, shadow=True, ncol=1)
    else:
        ax.legend(handles=patches, loc="lower right")
        ax.legend(handles=patches, bbox_to_anchor=(1.01,0.95), loc="upper left", fontsize=fontsize,
          fancybox=True, shadow=True, ncol=1, borderaxespad=0.)
        
def plot_crs(gal_name, axx=None, savepath=False, legend=True, is_arcsec=False, verbose=False, mode='box', r25=None):
    
    if axx is None:
        fig = plt.figure(figsize=[12,4.])
        ax = plt.gca()
    else:
        ax = axx
        
    ylim = 0
    xlim = 0
    
    ylim_ = 1e10
    xlim_ = 1e10
    
    mets = []
    cc = 0 
    
    altname = Ned.query_object(gal_name)['Object Name']
    cr_ = []
    cr_r_ = []
    bars = []
    for ind, row in df[df['obj_name']==altname[0]].iterrows():         
        method = row['method'] 
        cr_.append(float(row['cr']))
        cr_r_.append(float(row['err_cr'].split(',')[1][:-1]))
        bars.append(row['cr_bar'])
    
        cr=float(row['cr'])
        # print(row['cr_range'].split(',')[0][1:])
        cr_r = float(row['err_cr'].split(',')[1][:-1])
        cr_l = float(row['err_cr'].split(',')[0][1:])
        
        mets.append(method)

        if verbose:
            print(info, method)


        if method == 'SNs':
            ax.scatter(cr, 50, 100, color=method_color[method], label=method)
#           ax.scatter(cr, 50, 100, color=method_color[method], label=method, marker='+')
        else:
            if mode == 'box':
                ax.axvspan(cr-abs(cr_l),  cr+cr_r, alpha=0.5, color=method_color[method], label=method,linewidth = 1)
            else:
                ax.fill_between([cr-abs(cr_l),  cr], [0, 100.], alpha=0.4, color=method_color[method], label=method,linewidth = 3)
                ax.fill_between([cr,  cr+cr_r], [100, 0.], alpha=0.4, color=method_color[method], label=method,linewidth = 3)

            ax.scatter(cr, 0, 100, color=method_color[method], alpha=1, edgecolor='k',linewidth = 2)
            ax.scatter(cr, 100, 100, color=method_color[method], alpha=1, edgecolor='k',linewidth = 2)
            cc+=1
      
    if verbose:
            print('===='*30)
    if len(cr_)==0:
        if verbose:
            print(f'Gal {gal_name} not in entry {ind} ({row["method"]} / {row["link"]}), continue...')
            
    if legend:
        plot_cr_colorlegend(gal_name, ax, loc='lower right', fontsize=21, used_methods=mets, bars=bars)   
    
    if verbose:
           print(f'Gal {gal_name} IN entry {ind} ({row["method"]} / {row["link"]}):')
    
    
    ax.set_title(altname[0]+', number of values: '+str(len(cr_)), fontsize=30,pad=28)
    ax.set_xlabel(r'$R$, arcsec', fontsize=33)
    ax.set_yticks([])
    ax.set_facecolor('white')
        
    ax.set_ylim(0, 100)
    ax.set_xlim(0, max(cr_+cr_r_)*1.2)
    ax.tick_params(axis='both', which='major', labelsize=22)
        
        
    if r25 is not None:
        
        def to_new_axis(x):
            return x/r25
        
        def from_new_axis(x):
            return x*r25
        
        secax = ax.secondary_xaxis('top', functions=(to_new_axis, from_new_axis))
        secax.tick_params(axis='both', which='major', labelsize=18)
        secax.set_xlabel(r'$R/r_{25}$', fontsize=22)
        secax.xaxis.set_label_coords(1.05, 2)
        # ax.set_xlim(-1,np.max(np.max(np.array(cr_)+np.array(cr_r)),r25)+10)
    if savepath:
        plt.tight_layout()
        plt.savefig('./../CR_distributions/' + gal_name + '.png',transparent = False, dpi=300, bbox_inches='tight', pad_inches=0.05)
        plt.close(fig)

In [6]:
df = pd.read_csv('./../total_crs.csv',index_col=0) 

gf = df.groupby('obj_name')[['cr', 'r_25']].aggregate(list)
gf['cr_number'] = [len(row) for row in gf['cr'].values]

gff = gf[gf['cr_number']>1]
object_df = gff.index.values

In [7]:
len(object_df)

300

In [10]:
df = pd.read_csv('./../total_crs.csv',index_col=1) 
warnings.filterwarnings('ignore')

for ind,name in enumerate(object_df[200:]):
    
    try:
        r25_=None
        altname = Ned.query_object(name)['Object Name'][0]    
        for ind, row in df[df['obj_name']==altname].iterrows():
            r25_=row['r_25']
       # print(name,r25_)

        plot_crs(name, axx=None, savepath=True, legend=True, is_arcsec=True, verbose=False, mode=None, r25=r25_)
    except:
        plot_crs(name, axx=None, savepath=True, legend=True, is_arcsec=True, verbose=False, mode=None, )
        print(ind,name)